SETUP AND INSTALLATIONS

In [1]:
!pip install torch torchvision numpy matplotlib scikit-learn opencv-python tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cv2


In [3]:
dataset_path = "/content/drive/MyDrive/dataset-resized"

mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])

dataset = datasets.ImageFolder(root=dataset_path, transform=train_transform)
print("Classes:", dataset.classes, "Total:", len(dataset))


Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'] Total: 2527


🧠 RWC-Net (Fusion Model)

In [11]:
# Pretrained backbones
densenet = models.densenet201(pretrained=True)
mobilenet = models.mobilenet_v2(pretrained=True)

densenet_features = nn.Sequential(*list(densenet.children())[:-1], nn.AdaptiveAvgPool2d((1, 1)))
mobilenet_features = nn.Sequential(*list(mobilenet.children())[:-1], nn.AdaptiveAvgPool2d((1, 1)))

class RWCNet(nn.Module):
    def __init__(self, num_classes=6):
        super(RWCNet, self).__init__()
        self.densenet = densenet_features
        self.mobilenet = mobilenet_features

        # Freeze backbone parameters
        for param in self.densenet.parameters():
            param.requires_grad = False
        for param in self.mobilenet.parameters():
            param.requires_grad = False

        self.fc = nn.Sequential(
            nn.Linear(1920 + 1280, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

        self.aux1 = nn.Linear(1920, num_classes)
        self.aux2 = nn.Linear(1280, num_classes)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        d_out = torch.flatten(self.densenet(x), 1)
        m_out = torch.flatten(self.mobilenet(x), 1)

        combined = torch.cat((d_out, m_out), dim=1)
        out = self.fc(combined)

        aux_out1 = self.aux1(d_out)
        aux_out2 = self.aux2(m_out)

        return self.log_softmax(out), aux_out1, aux_out2

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most u

⚙️ 5-Fold Cross-Validation Training

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

def train_one_fold(train_idx, val_idx, fold, epochs=5, batch_size=16, lr=1e-5):
    print(f"\n===== Fold {fold+1} =====")

    train_ds = Subset(dataset, train_idx)
    val_ds = Subset(dataset, val_idx)
    train_ds.dataset.transform = train_transform
    val_ds.dataset.transform = val_transform

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

    model = RWCNet(num_classes=len(dataset.classes)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0,0
        for x,y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            x,y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)

            loss = criterion(out,y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()*x.size(0)
            train_correct += (out.argmax(1)==y).sum().item()

        val_loss, val_correct = 0,0
        model.eval()
        with torch.no_grad():
            for x,y in val_loader:
                x,y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out,y)
                val_loss += loss.item()*x.size(0)
                val_correct += (out.argmax(1)==y).sum().item()

        print(f"Epoch {epoch+1} | Train Acc: {train_correct/len(train_ds):.3f} | Val Acc: {val_correct/len(val_ds):.3f}")

    return model

# Run 5 folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(np.arange(len(dataset)))):
    model = train_one_fold(train_idx, val_idx, fold, epochs=5)


===== Fold 1 =====


Epoch 1/5: 100%|██████████| 127/127 [10:27<00:00,  4.94s/it]


Epoch 1 | Train Acc: 0.347 | Val Acc: 0.506


Epoch 2/5: 100%|██████████| 127/127 [00:24<00:00,  5.29it/s]


Epoch 2 | Train Acc: 0.517 | Val Acc: 0.650


...

Epoch 3 | Train Acc: 0.626 | Val Acc: 0.703


Epoch 4/5: 100%|██████████| 127/127 [00:24<00:00,  5.18it/s]


Epoch 4 | Train Acc: 0.690 | Val Acc: 0.721


Epoch 5/5: 100%|██████████| 127/127 [00:24<00:00,  5.21it/s]


Epoch 5 | Train Acc: 0.715 | Val Acc: 0.723
